<a href="https://colab.research.google.com/github/CassandraMzola/PortfolioProjects/blob/main/earthquake_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from google.colab import auth
auth.authenticate_user()


In [19]:
!pip install --upgrade google-cloud-bigquery


In [20]:
from google.cloud import bigquery

client = bigquery.Client(project='earthquake_project')


In [21]:
import requests
import pandas as pd
from google.cloud import bigquery

# Pull JSON data from USGS
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
response = requests.get(url)
data = response.json()

# Flatten JSON
earthquakes = pd.json_normalize(data['features'])

# Keep important columns
earthquakes = earthquakes[['properties.time','properties.place','properties.mag','geometry.coordinates']]

# Split coordinates
earthquakes[['longitude','latitude','depth']] = pd.DataFrame(
    earthquakes['geometry.coordinates'].tolist(), index=earthquakes.index
)

# Drop original column & rename
earthquakes.drop(columns=['geometry.coordinates'], inplace=True)
earthquakes.rename(columns={
    'properties.time': 'time',
    'properties.place': 'place',
    'properties.mag': 'magnitude'
}, inplace=True)

# Convert timestamp to datetime
earthquakes['time'] = pd.to_datetime(earthquakes['time'], unit='ms')

print(earthquakes.head())


                     time                         place  magnitude  longitude  \
0 2025-08-22 09:33:55.850    4 km SW of Volcano, Hawaii       2.06 -155.26683   
1 2025-08-22 09:02:42.241  59 km SW of Cantwell, Alaska       1.60 -149.77790   
2 2025-08-22 08:59:46.950   2 km SE of Lake Henshaw, CA       1.37 -116.74450   
3 2025-08-22 08:58:53.210      15 km SE of Ocotillo, CA       1.16 -115.88900   
4 2025-08-22 08:56:17.962    3 km ENE of Willow, Alaska       1.10 -149.98020   

    latitude  depth  
0  19.416000   1.72  
1  63.015800  69.70  
2  33.229167   9.29  
3  32.637833   9.64  
4  61.756300  33.80  


In [22]:
# Initialize BigQuery client
client = bigquery.Client(project='earthquake_project')
dataset_id = 'earthquake_dataset'
table_id = 'earthquakes'


In [23]:
!pip install --upgrade pandas-gbq
from pandas_gbq import to_gbq


In [28]:
dataset_id = 'earthquake_dataset'
table_id = 'earthquakes'
table_ref = f"{dataset_id}.{table_id}"
project_id = 'earthquake-project-469810'


In [29]:
to_gbq(
    dataframe=earthquakes,
    destination_table= f"{dataset_id}.{table_id}",
    project_id= 'earthquake-project-469810',
    if_exists='append'
)
print("Upload complete!")



100%|██████████| 1/1 [00:00<00:00, 10922.67it/s]

Upload complete!
